## Find each Franchise's rostered players

In [1]:
# username and passwords
league_id = "53906"

In [2]:
# import dependencies
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import date
import plotly.express as px
import plotly.graph_objects as go

## Pull Franchise data from MyFantasyLeague

In [3]:
# Get all league data
urlString = "https://www54.myfantasyleague.com/2022/export?TYPE=league&L=53906"
response = requests.get(urlString)
soup = BeautifulSoup(response.content,'xml')

data = []
franchises = soup.find_all('franchise')
for i in range(len(franchises)):
    rows = [franchises[i].get("id"), franchises[i].get("name")]
    data.append(rows)
franchise_df = pd.DataFrame(data)
franchise_df.columns=['FranchiseID','FranchiseName']
franchise_df = franchise_df.append({"FranchiseID":"FA", "FranchiseName":"Free Agent"}, ignore_index=True)
franchise_df

,FranchiseID,FranchiseName
0,0001,Providence Monastic Spirits
1,0002,Washington Fantasy Football Team
2,0003,Verdansk Vaqueros FFC
3,0004,Down with the Sickness unto Death
4,0005,Crocodilopolis Body Snatchers
5,0006,O'Henry's Stiffies
6,0007,Comeback Kings
7,0008,WHAM! BAM! Thank You CAM!
8,0009,Pretty Big Wieners
9,0010,Saskatoon Squatches


## Pull player data from MyFantasyLeague

In [4]:
# Get all players' name, team name, position
urlString = "https://api.myfantasyleague.com/2022/export?TYPE=players"
response = requests.get(urlString)
soup = BeautifulSoup(response.content,'xml')

data = []
players = soup.find_all('player')
for i in range(len(players)):
    rows = [players[i].get("id"), players[i].get("name"), players[i].get("position"), players[i].get("team")]
    data.append(rows)
player_df = pd.DataFrame(data)
player_df.columns=['PlayerID','Name', 'Position', 'Team']


In [5]:
# Get franchise rosters
urlString = "https://www54.myfantasyleague.com/2022/export?TYPE=rosters&L=53906"
response = requests.get(urlString)
soup = BeautifulSoup(response.content,'xml')

data = []
franchises = soup.find_all('franchise')
for i in range(0,len(franchises)):
    current_franchise = franchises[i].find_all('player')
    for j in range(0,len(current_franchise)):
        rows = [franchises[i].get("id"), franchises[i].get("week"), current_franchise[j].get("id"), current_franchise[j].get("status")]
        data.append(rows)
rosters_df = pd.DataFrame(data)

# Get Free Agents
urlString = "https://www54.myfantasyleague.com/2022/export?TYPE=freeAgents&L=53906"
response = requests.get(urlString)
soup = BeautifulSoup(response.content,'xml')

data = []
freeAgents = soup.find_all('player')
for i in range(len(freeAgents)):
    rows = ["FA", "", freeAgents[i].get("id"), "Free Agent"]
    data.append(rows)
fa_df = pd.DataFrame(data)
rosters_df = rosters_df.append(fa_df)
rosters_df.columns=['FranchiseID','Week','PlayerID','RosterStatus']



In [6]:
# Get Shark Ranks
urlString = "https://api.myfantasyleague.com/2022/export?TYPE=playerRanks"
response = requests.get(urlString)
soup = BeautifulSoup(response.content,'xml')

data = []
sharkRanks = soup.find_all('player')
for i in range(len(sharkRanks)):
    rows = [sharkRanks[i].get("id"), sharkRanks[i].get("rank")]
    data.append(rows)
shark_df = pd.DataFrame(data)
shark_df.columns=['PlayerID','SharkRank']
shark_df['SharkRank'] = shark_df['SharkRank'].astype('int32')


In [7]:
# Get adp
urlString = "https://api.myfantasyleague.com/2022/export?TYPE=adp"
response = requests.get(urlString)
soup = BeautifulSoup(response.content,'xml')

data = []
players = soup.find_all('player')
for i in range(len(players)):
    rows = [players[i].get("id"), players[i].get("averagePick")]
    data.append(rows)
adp_df = pd.DataFrame(data)
adp_df.columns=['PlayerID','ADP']
adp_df['ADP'] = adp_df['ADP'].astype('float32')



In [8]:
# Merge all dfs
complete = player_df.merge(rosters_df, on='PlayerID', how='left').merge(franchise_df[['FranchiseID', 'FranchiseName']], on='FranchiseID', how='left').merge(shark_df, on='PlayerID', how='left').merge(adp_df, on='PlayerID', how='left')
complete['FranchiseID'].fillna("FA", inplace=True)
complete['FranchiseName'].fillna("Free Agent", inplace=True)
complete['RosterStatus'].fillna("Free Agent", inplace=True)
complete['SharkRank'].fillna(3000, inplace=True)
complete['ADP'].fillna(3000, inplace=True)
complete = complete.sort_values(by=['SharkRank'])
complete.reset_index(inplace=True, drop=True)

In [9]:
qbs_2022 = complete[complete['Position'] == "QB"]
qbs_2022.reset_index(inplace=True, drop=True)
rbs_2022 = complete[complete['Position'] == "RB"]
rbs_2022.reset_index(inplace=True, drop=True)
wrs_2022 = complete[complete['Position'] == "WR"]
wrs_2022.reset_index(inplace=True, drop=True)
tes_2022 = complete[complete['Position'] == "TE"]
tes_2022.reset_index(inplace=True, drop=True)
pks_2022 = complete[complete['Position'] == "PK"]
pks_2022.reset_index(inplace=True, drop=True)
defs_2022 = complete[complete['Position'] == "Def"]
defs_2022.reset_index(inplace=True, drop=True)

## Import standard relative values

In [10]:
qbs = pd.read_excel("data/RelativeValues.xlsx", sheet_name="QB")
rbs = pd.read_excel("data/RelativeValues.xlsx", sheet_name="RB")
wrs = pd.read_excel("data/RelativeValues.xlsx", sheet_name="WR")
tes = pd.read_excel("data/RelativeValues.xlsx", sheet_name="TE")
pks = pd.read_excel("data/RelativeValues.xlsx", sheet_name="PK")
defs = pd.read_excel("data/RelativeValues.xlsx", sheet_name="DEF")

qbs = qbs[['Projection_Relative', "Projection_Absolute"]]
rbs = rbs[['Projection_Relative', "Projection_Absolute"]]
wrs = wrs[['Projection_Relative', "Projection_Absolute"]]
tes = tes[['Projection_Relative', "Projection_Absolute"]]
pks = pks[['Projection_Relative', "Projection_Absolute"]]
defs = defs[['Projection_Relative', "Projection_Absolute"]]


In [11]:
# Merge dfs
qbs_2022 = pd.merge(qbs_2022, qbs, how="left", left_index=True, right_index=True)
rbs_2022 = pd.merge(rbs_2022, rbs, how="left", left_index=True, right_index=True)
wrs_2022 = pd.merge(wrs_2022, wrs, how="left", left_index=True, right_index=True)
tes_2022 = pd.merge(tes_2022, tes, how="left", left_index=True, right_index=True)
pks_2022 = pd.merge(pks_2022, pks, how="left", left_index=True, right_index=True)
defs_2022 = pd.merge(defs_2022, defs, how="left", left_index=True, right_index=True)
analyzed = pd.concat([qbs_2022, rbs_2022, wrs_2022, tes_2022, pks_2022, defs_2022])


In [12]:
analyzed.groupby('FranchiseName').sum().sort_values(by='Projection_Relative', ascending=False)

,SharkRank,ADP,Projection_Relative,Projection_Absolute
FranchiseName,,,,
Pretty Big Wieners,14731.0,2.023412e+04,776.270002,4076.970002
Washington Fantasy Football Team,25252.0,2.633182e+04,739.753322,4039.753322
Saskatoon Squatches,22357.0,2.032886e+04,674.503327,4024.503327
WHAM! BAM! Thank You CAM!,16792.0,1.459053e+04,582.796655,3501.196655
Comeback Kings,27413.0,3.249910e+04,557.773327,3712.673327
Crocodilopolis Body Snatchers,19252.0,2.099540e+04,556.736661,4088.736661
Verdansk Vaqueros FFC,24088.0,3.504817e+04,313.726668,3909.626668
Down with the Sickness unto Death,21273.0,2.635170e+04,210.029988,3504.029988
O'Henry's Stiffies,26517.0,2.415308e+04,197.703325,3656.703325


In [13]:
analyzed['Date'] = date.today()
priorWeeks = pd.read_csv("data/2022Values.csv", index_col=0)
priorWeeks['Date'] = pd.to_datetime(priorWeeks['Date'])
result = pd.concat([priorWeeks, analyzed], axis=0, ignore_index=True)
#result.to_csv("2022Values.csv")

## Roster-builder

In [14]:
today = result[result['Date'] == date.today()].sort_values(by='Projection_Relative', ascending=False, ignore_index=True)

In [15]:
qbs_rostered = today[today['Position'] == "QB"]
qbs_rostered.reset_index(inplace=True, drop=True)
rbs_rostered = today[today['Position'] == "RB"]
rbs_rostered.reset_index(inplace=True, drop=True)
wrs_rostered = today[today['Position'] == "WR"]
wrs_rostered.reset_index(inplace=True, drop=True)
tes_rostered = today[today['Position'] == "TE"]
tes_rostered.reset_index(inplace=True, drop=True)

In [16]:
qbs_top = qbs_rostered.sort_values(by='Projection_Relative', ascending=False, ignore_index=True).groupby('FranchiseName').head(1)
rbs_top = rbs_rostered.sort_values(by='Projection_Relative', ascending=False, ignore_index=True).groupby('FranchiseName').head(2)
wrs_top = wrs_rostered.sort_values(by='Projection_Relative', ascending=False, ignore_index=True).groupby('FranchiseName').head(3)
tes_top = tes_rostered.sort_values(by='Projection_Relative', ascending=False, ignore_index=True).groupby('FranchiseName').head(2)

qbs_remainder = qbs_rostered[~qbs_rostered['PlayerID'].isin(qbs_top['PlayerID'])].groupby('FranchiseName').head(1)
rbs_remainder = rbs_rostered[~rbs_rostered['PlayerID'].isin(rbs_top['PlayerID'])].groupby('FranchiseName').head(3)
wrs_remainder = wrs_rostered[~wrs_rostered['PlayerID'].isin(wrs_top['PlayerID'])].groupby('FranchiseName').head(3)
tes_remainder = tes_rostered[~tes_rostered['PlayerID'].isin(tes_top['PlayerID'])].groupby('FranchiseName').head(3)
                                                         
remainder = pd.concat([qbs_remainder, rbs_remainder, wrs_remainder, tes_remainder])
                             
top_remainders = remainder.sort_values(by='Projection_Absolute', ascending=False, ignore_index=True).groupby('FranchiseName').head(3)
                             
fran_rost = pd.concat([qbs_top, rbs_top, wrs_top, tes_top, top_remainders])
fran_rost = fran_rost.sort_values(by='Projection_Relative', ascending=False, ignore_index=True)


## Examine Data

In [17]:
fran_rank = fran_rost.groupby('FranchiseName').sum().sort_values(by='Projection_Relative', ascending=False)
fran_rank

,SharkRank,ADP,Regressed,Projection_Relative,Projection_Absolute
FranchiseName,,,,,
Washington Fantasy Football Team,842.0,661.139991,0.0,963.713328,2214.913328
Saskatoon Squatches,670.0,593.280004,0.0,949.956658,2201.156658
Verdansk Vaqueros FFC,1052.0,699.260006,0.0,889.796669,2148.396669
Crocodilopolis Body Snatchers,756.0,672.919998,0.0,889.203332,2144.103332
Pretty Big Wieners,1382.0,783.109995,0.0,826.336675,2081.236675
Gus The Bus,965.0,916.759987,0.0,754.199996,1970.799996
WHAM! BAM! Thank You CAM!,1426.0,1097.079990,0.0,720.099995,1974.999995
O'Henry's Stiffies,2406.0,847.500009,0.0,714.193320,1972.793320
Down with the Sickness unto Death,1454.0,1060.799997,0.0,691.689994,1946.589994


In [18]:
sorter = fran_rank.index

fran_rost.FranchiseName = fran_rost.FranchiseName.astype("category")
fran_rost.FranchiseName.cat.set_categories(sorter, inplace=True)
fran_rost.sort_values(["FranchiseName"], inplace=True)

fig = px.bar(fran_rost, 
             x="FranchiseName", 
             y="Projection_Relative", 
             color="Position", 
             text='Name', 
             color_discrete_map={
                    "RB": "#062647", #blue #1033a6 #0c2987 1033a6
                    "TE": "#43B3AE", #teal #02687b #038097 1295ad
                    "WR": "#621B74", #purple #4f22bc #643fc1 643fc1
                    "QB": "#ffa524"}, #gold #f5d000 f5d000
             category_orders={
                 "Position": ["RB", "QB", "WR", "TE"]}
            )
fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()

In [26]:
FranchiseSelection = ["Free Agent", "Crocodilopolis Body Snatchers"]
defs_2022[defs_2022['FranchiseName'].isin(FranchiseSelection)].sort_values(by="Projection_Relative", ascending=False).head(50)

,PlayerID,Name,Position,Team,FranchiseID,Week,RosterStatus,FranchiseName,SharkRank,ADP,Projection_Relative,Projection_Absolute
3,0530,"49ers, San Francisco",Def,SFO,0005,1,ROSTER,Crocodilopolis Body Snatchers,529.0,151.490005,83.680000,200.680000
15,0507,"Browns, Cleveland",Def,CLE,0005,1,ROSTER,Crocodilopolis Body Snatchers,804.0,3000.000000,47.679998,164.679998
23,0511,"Broncos, Denver",Def,DEN,0005,1,ROSTER,Crocodilopolis Body Snatchers,995.0,173.080002,2.973333,119.973333
